In [1]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg, FQ
import numpy as np
import galois   

In [2]:
GF = galois.GF(curve_order)
# takes about 2 minutes

In [3]:
# R1CS

x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)

L = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
    ]))

R = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

O = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [curve_order-2,1,0,3,0,curve_order-1,0,curve_order-2] # 113 - 2 = 111
    ]))

witness = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))
print(f"witness= {type(witness)}")
print(f"generated L, R, O R1CS matrices and witness vector")
print(f"verifying that L.w * R.w == O.w")
result = O.dot(witness) == np.multiply(L.dot(witness),R.dot(witness))
assert result.all(), "result contains an inequality"
print(f"verified")

witness= <class 'galois.GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)'>
generated L, R, O R1CS matrices and witness vector
verifying that L.w * R.w == O.w
verified


In [38]:
# R1CS => QAP
target_coeffs = GF(np.array([1,2,3,4,5]))
target_polynomial = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4,5])).astype(int),  field=GF)

def matrix_to_vector_of_polynomials(matrix):
    return np.apply_along_axis(interpolate_columns, 0, matrix)

def interpolate_columns(column):
    return galois.lagrange_poly(target_coeffs, column)

# Once interpolation is finished, L, R, O = U.a, V.a, W.a
U_vector_of_poly_x = matrix_to_vector_of_polynomials(L)
V_vector_of_poly_x = matrix_to_vector_of_polynomials(R)
W_vector_of_poly_x = matrix_to_vector_of_polynomials(O)
# print(U_vector_of_poly_x)

def aggregate_vector_of_polynomials(vector_of_poly, witness):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        sum += vector_of_poly[i]*witness[i]
    return sum

# This is a polynomial of the inner product of Matrix and Witness
Ua = aggregate_vector_of_polynomials(U_vector_of_poly_x, witness)
Va = aggregate_vector_of_polynomials(V_vector_of_poly_x, witness)
Wa = aggregate_vector_of_polynomials(W_vector_of_poly_x, witness)

print(f"Ua= {Ua}")

# We can now create h(x)t(x)
hx, remainder = divmod(((Ua*Va) - Wa), target_polynomial)
assert remainder == 0, "remainder is not 0"
Ua * Va == Wa + hx * target_polynomial 
print(f"lhs = rhs so they are the same")

tau = GF(9)
Ua(tau) * Va(tau) == Wa(tau) + hx(tau) * target_polynomial(tau)
print(f"when evaluated at random number tau, theyare the same")


Ua= 7296080957279758407415468581752425029516121466805344781232734728858602831849x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301416197x^3 + 3648040478639879203707734290876212514758060733402672390616367364429301414937x^2 + 7296080957279758407415468581752425029516121466805344781232734728858602833414x + 21888242871839275222246405745257275088548364400416034343698204186575808494842
lhs = rhs so they are the same
when evaluated at random number tau, theyare the same


In [56]:
# Trusted Setup
from functools import reduce
tau = GF(9)
powers_of_tau_G1 = [multiply(G1, int(tau) ** i) for i in range(Ua.degree + 1)]
powers_of_tau_G2 = [multiply(G2, int(tau) ** i) for i in range(Ua.degree + 1)]
t_of_tau = target_polynomial(tau)
target_powers_of_tau_G1 = [multiply(G1, int(tau) ** i * int(t_of_tau)) for i in range(target_polynomial.degree)]

# Prover evaluate QAP at Tau
# This is not encrypted
Ua_tau = Ua(tau)
Va_tau = Va(tau)
Wa_tau = Wa(tau)
hx_tau = hx(tau)
tx_tau = target_polynomial(tau)
assert Ua_tau * Va_tau == Wa_tau + hx_tau * tx_tau, "Ua_tau * Va_tau != Wa_tau + hx_tau * tx_tau"

def inner_product(powers_of_tau, coeffs):
    # print(len(powers_of_tau), len(coeffs))
    # print(f"scaling {coeffs[0]} with {powers_of_tau[0]} ")
    # print(f"scaling {coeffs[1]} with {powers_of_tau[1]} ")
    # print(f"scaling {coeffs[2]} with {powers_of_tau[2]} ")
    # print(f"scaling {coeffs[3]} with {powers_of_tau[3]} ")
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(powers_of_tau, coeffs)), Z1)

Ua_tau_G1 = inner_product(powers_of_tau_G1, Ua.coeffs[::-1])
Va_tau_G2 = inner_product(powers_of_tau_G2, Va.coeffs[::-1])
Wa_tau_G1 = inner_product(powers_of_tau_G1, Wa.coeffs[::-1])
# target_powers_of_tau_G1
ht_tau = inner_product(target_powers_of_tau_G1, hx.coeffs[::-1])
print(ht_tau)

lhs = pairing(Va_tau_G2, Ua_tau_G1)
rhs = pairing(G2, add(Wa_tau_G1, ht_tau))

if eq(lhs, rhs):
    print("verified")
else:
    print("not verified")

(10229578500132525219804232293477008740220480719254688155636045124922735668252, 9284168382991294717218307925110778314947151538513504999199960077137521427660)
verified


In [62]:
# shift lhs and rhs by alpha and beta
# Trusted Setup
alpha = GF(2)
beta = GF(3)
AlphaG1 = multiply(G1, int(alpha))
BetaG2 = multiply(G2, int(beta))

# encrypted evaluation: shifts the U and V polynomial by alpha and beta, evaluates at powers of tau
powers_of_C_G1 = []
for i in range(len(U_vector_of_poly_x)):
    print(f"i= {i}, tau**i= {tau**i}")
    u_eval_tau_i_beta = beta * U_vector_of_poly_x[i](tau**i)
    v_eval_tau_i_alpha = alpha * V_vector_of_poly_x[i](tau**i)
    w_eval_tau_i = W_vector_of_poly_x[i](tau**i)
    powers_of_C_G1.append(multiply(G1, int(u_eval_tau_i_beta + v_eval_tau_i_alpha + w_eval_tau_i)))



print(f"powers_of_C_G1= {powers_of_C_G1}")

Cprime = reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(powers_of_C_G1, witness)), Z1)
C_new = add(Cprime, ht_tau)

A_new = add(AlphaG1, Ua_tau_G1)
B_new = add(BetaG2, Va_tau_G2)

lhs = pairing(B_new, A_new)
rhs_1 = pairing(BetaG2,AlphaG1)
rhs_2 = pairing(G2,C_new)
rhs = rhs_1 * rhs_2

if eq(lhs, rhs):
    print("verified")
else:
    print("not verified")


i= 0, tau**i= 1
i= 1, tau**i= 9
i= 2, tau**i= 81
i= 3, tau**i= 729
i= 4, tau**i= 6561
i= 5, tau**i= 59049
i= 6, tau**i= 531441
i= 7, tau**i= 4782969
powers_of_C_G1= [None, (1498339796778307471365624041433540006436600402203263068984044382637138627232, 16602075090262163391842937953477689188383027283447549643521873518378054630844), (19713731097331214556946270407996655093938624249746801813868484970177446250827, 7915450358416394160244865461310067414026740581724236031233817127847290564398), (18802409903743233850598038877913051077699318360879489910323138544374008671560, 6018002251046286340202980904321527968652906953492375385565894913798893110620), (16749259040035862000161139699914125634044344427748873833069450502986132926775, 21228218326893656881892551613231272194089486779098148509380441788503869847289), (4256573437536214784753561414244738330606124694572745625878333704018012795384, 4840905012468090481208016870804876535614827290217972219136190742298558358708), (28155223736805382834998105212175